# Recommendation Systems Exhibition

In [ ]:
# Import packages used throughout
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

from data import import_data 
from methods import utils, losses, matrix_factorization, AutoRec #, NeuMF, Caser

## MovieLens Dataset

In [ ]:
# Import data
names = ["user_id", "item_id", "rating", "timestamp"]
data = pd.read_csv("data/u.data", delimiter='\t', names = names, engine = "python")
num_users = data.user_id.unique().shape[0]
num_items = data.item_id.unique().shape[0]

## Matrix Factorization

In [ ]:
# Obtain the data
num_users, num_items, train_iter, test_iter = import_data.split_and_load_ml100k(data,
    num_users, num_items, test_ratio = 0.1, batch_size=512)

In [ ]:
# Define evaluator
def evaluator(net, test_iter):
    rmse_list = []
    for idx, (users, items, ratings) in enumerate(test_iter):
        r_hat = net(users, items)
        rmse_value = torch.sqrt(((r_hat - ratings)**2).mean())
        rmse_list.append(float(rmse_value))

    return np.mean(np.array(rmse_list))

In [ ]:
# Prepare model
lr, num_epochs, wd = 0.002, 20, 1e-5

mf_net = matrix_factorization.MF(30, num_users, num_items)
optimizer = optim.Adam(mf_net.parameters(), lr = lr, weight_decay = wd)

In [ ]:
# Train and evaluate model
rmse_list = []
loss_list = []
for epoch in range(num_epochs):
    total_loss, num_samples = 0, 0

    # Train each batch
    mf_net.train()
    for i, (users, items, ratings) in enumerate(train_iter):
        optimizer.zero_grad()

        predictions = mf_net(users, items)
        output = ((predictions - ratings)**2).mean()
        output.backward()
        optimizer.step()

        total_loss += output
        num_samples += users.shape[0]
    
    # Evaluate
    mf_net.eval()
    rmse = evaluator(mf_net, test_iter)
    rmse_list.append(rmse)
    loss_list.append(total_loss/num_samples)

    print(f"Epoch {epoch}:\n\tloss = {total_loss/num_samples}\n\trmse = {rmse}")

In [ ]:
# Visualize
fig, ax = plt.subplots(2)
fig.suptitle(f"Matrix Factorization Test over {num_epochs} Epochs")

x_vals = list(range(1, num_epochs + 1))
ax[0].plot(x_vals, rmse_list, label = "RMSE", color = "blue")
ax[0].set_ylabel("RMSE")
ax[0].set_xticks(x_vals[::2])
ax[0].legend()

ax[1].plot(x_vals, loss_list, label = "Loss", color = "green")
ax[1].set_ylabel("Loss")
ax[1].set_xlabel("Epoch")
ax[1].set_xticks(x_vals[::2])
ax[1].legend()

## AutoRec

In [ ]:
# Obtain the data
train_data, test_data = import_data.split_data_ml100k(data, num_users, num_items)

_, _, _, train_inter_matrix = import_data.load_data_ml100k(train_data, num_users, num_items)
_, _, _, test_inter_matrix = import_data.load_data_ml100k(test_data, num_users, num_items)

train_inter_tensor = torch.from_numpy(train_inter_matrix).to(torch.float)
test_inter_tensor = torch.from_numpy(test_inter_matrix).to(torch.float)

train_iter = DataLoader(train_inter_tensor, shuffle = True, batch_size = 256,
    num_workers = 4)
test_iter = DataLoader(test_inter_tensor, shuffle = False, batch_size = 1024,
    num_workers = 4)

In [ ]:
# Define evaluator
def evaluator(network, test_iter):
    rmse_list = []
    for idx, users_ratings in enumerate(test_iter):
        recons = network(users_ratings)
        rmse_value = torch.sqrt(
            ((torch.sign(users_ratings) * recons - users_ratings)**2).mean())
        rmse_list.append(float(rmse_value))

    return np.mean(np.array(rmse_list))

In [ ]:
# Prepare model
lr, num_epochs, wd = 0.002, 25, 1e-5

autorec_net = AutoRec(500, num_users)
optimizer = optim.Adam(autorec_net.parameters(), lr = lr, weight_decay=wd)

In [ ]:
# Train and evaluate model
rmse_list = []
loss_list = []
for epoch in range(num_epochs):
    total_loss, num_samples = 0, 0

    # Train each batch
    autorec_net.train()
    for i, user_ratings in enumerate(train_iter):
        optimizer.zero_grad()

        predictions = autorec_net(user_ratings)
        output = ((predictions - user_ratings)**2).mean()
        output.backward()
        optimizer.step()

        total_loss += output
        num_samples += user_ratings.shape[0]
    
    # Evaluate
    autorec_net.eval()
    rmse = evaluator(autorec_net, test_iter)
    rmse_list.append(rmse)
    loss_list.append(total_loss/num_samples)

    print(f"Epoch {epoch}:\n\tloss = {total_loss/num_samples}\n\trmse = {rmse}")

In [ ]:
# Visualize
fig, ax = plt.subplots(2)
fig.suptitle(f"AutoRec Test over {num_epochs} Epochs")

x_vals = list(range(1, num_epochs + 1))
ax[0].plot(x_vals, rmse_list, label = "RMSE", color = "blue")
ax[0].set_ylabel("RMSE")
ax[0].set_xticks(x_vals[::2])
ax[0].legend()

ax[1].plot(x_vals, loss_list, label = "Loss", color = "green")
ax[1].set_ylabel("Loss")
ax[1].set_xlabel("Epoch")
ax[1].set_xticks(x_vals[::2])
ax[1].legend()

## NeuMF

In [ ]:
# Split Dataset
batch_size = 1024
train_data, test_data = split_data_ml100k(data, num_users, num_items,'seq-aware')
users_train, items_train, ratings_train, candidates = load_data_ml100k(
    train_data, num_users, num_items, feedback="implicit")
users_test, items_test, ratings_test, test_iter = load_data_ml100k(
    test_data, num_users, num_items, feedback="implicit")
train_iter = gluon.data.DataLoader(
    PRDataset(users_train, items_train, candidates, num_items ), batch_size,
    True, last_batch="rollover", num_workers=get_dataloader_workers())

In [ ]:
# Create and initialize model
devices = try_all_gpus()
net = NeuMF(10, num_users, num_items, nums_hiddens=[10, 10, 10])
net.initialize(ctx=devices, force_reinit=True, init=mx.init.Normal(0.01))

In [ ]:
# Train model
lr, num_epochs, wd, optimizer = 0.01, 8, 1e-5, 'adam'
loss = BPRLoss()
hit_rate_list_neu, auc_list_neu = train_ranking(net, train_iter, test_iter, loss, trainer,
                          None, num_users, num_items, num_epochs, devices, evaluate_ranking, candidates)

## Caser

In [ ]:
# Load dataset
TARGET_NUM, L, batch_size = 1, 5, 4096
train_data, test_data = split_data_ml100k(data, num_users, num_items,
                                              'seq-aware')
users_train, items_train, ratings_train, candidates = load_data_ml100k(
    train_data, num_users, num_items, feedback="implicit")
users_test, items_test, ratings_test, test_iter = load_data_ml100k(
    test_data, num_users, num_items, feedback="implicit")
train_seq_data = SeqDataset(users_train, items_train, L, num_users,
                            num_items, candidates)
train_iter = gluon.data.DataLoader(train_seq_data, batch_size, True,
                                   last_batch="rollover",
                                   num_workers=get_dataloader_workers())
test_seq_iter = train_seq_data.test_seq
train_seq_data[0]

In [ ]:
devices = try_all_gpus()
net = Caser(10, num_users, num_items, L)
net.initialize(ctx=devices, force_reinit=True, init=mx.init.Normal(0.01))
lr, num_epochs, wd, optimizer = 0.04, 8, 1e-5, 'adam'
loss = BPRLoss()
trainer = gluon.Trainer(net.collect_params(), optimizer,
                        {"learning_rate": lr, 'wd': wd})

hit_rate_list_caser, auc_list_caser = train_ranking(net, train_iter, test_iter, loss, trainer, test_seq_iter, num_users, num_items, num_epochs, devices, evaluate_ranking, candidates, eval_step=1)